# 第6回 その2: ロジスティック回帰を用いた教師ありクラスタリング
06_01_linear_discriminant_analysis.ipynbと同じデータに対して，ロジスティック回帰を用いた教師ありクラスタリングを行います。

## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/06"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/06/
# 現在のフォルダの中身を表示
%ls

`2class_data.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込みと標準化
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`2class_data.csv` を読み込みます。  
このデータx1とx2の二次元データで，クラス0とクラス1の2種類のクラスのどちらかに属しています。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('2class_data.csv', encoding='SHIFT-JIS')

# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# x1とx2のデータを抽出し，numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:, ['x1','x2']].to_numpy()

# クラス番号を抽出し，numpy用データ(ndarray型) に変換する。
Y = csv_data.loc[:,'class'].to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

# クラス数を得る。
num_classes = int(np.max(Y) + 1)
print('Number of classes: ' + str(num_classes))

標準化を行い，データをプロットします。

In [ ]:
# データを標準化する。
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# データをプロット
plt.figure(figsize=(7,7))
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class0')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

## ステップ2: ロジスティック回帰
ロジスティック回帰を行います。  
二次元のデータを${\bf x} = [x_1, x_2]$としたとき，ロジスティック回帰は以下の式になります。  
$\hat{y} = \frac{1}{1+e^{-z}}$  
$z = w_0 + w_1x_1 + w_2x_2$  
$z$の式については，${\bf w} = [w_0, w_1, w_2]$，${\bf x}_{\rm mod} = [1, x_1, x_2]^{T}$と置くことで，  
$z = {\bf w}{\bf x}_{\rm mod}$  
とも書けます。  以下のソースコードではこの書き方を使うことで，実装をシンプルにしています。   

まず，$\bf w$ の初期値を決めます。  
w_0，つまり切片の初期値はゼロとし，w_1以降，つまりデータの各次元に関する重みの成分は正規分布に従う乱数で決めます。

In [ ]:
np.random.seed(0)

initial_w = np.append(0, np.random.randn(num_dimensions))

print('w = ' + str(initial_w))

初期値における境界線をプロットします。  
境界線は$z = w_0 + w_1x_1 + w_2x_2 = 0$ですので，  
$x_2 = -(w_0 + w_1x_1) / w_2$  
と変形したものをプロットします。  
(正確には境界線は$\hat{y} = \frac{1}{1+e^{-z}} = 0.5$ですが，$\hat{y}$が0.5となるのは$z$が0のときなので，実質的な境界線は$z = 0$となります。)

In [ ]:
# データをプロット
x1 = np.linspace(-3, 3)
x2 = -1.0 * (initial_w[0] + initial_w[1]*x1) / initial_w[2]
plt.figure(figsize=(7,7))
plt.plot(x1, x2)
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class1')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class2')
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

初期値ですので，正確に分離できていない境界線となっています。  
例として，この状態で一度ロジスティック回帰を実行してみましょう。

1サンプルデータに対してロジスティック回帰を行う関数を以下に定義します。  

シグモイド関数の定義  
${\rm sigmoid}(z) = \frac{1}{1+e^{-z}}$  
ロジスティック回帰の定義  
$\hat{y} = sigmoid(z)$  
$z = w_0 + w_1x_1 + w_2x_2 + \dots = [w_0, w_1, w_2, \dots][1, x_1, x_2, \dots]^{T}$  

In [ ]:
def sigmoid(z):
  '''
      シグモイド関数
  '''
  return (1+np.exp(-1*z))**(-1)

def logistic_regression(x, w):
  '''
      1サンプルデータに対してロジスティック回帰を行う
      x: 1サンプルデータ。要素数=次元数のベクトル
      w: 切片と各次元に対する重み。要素数=(次元数+1)のベクトル(+1は切片の分)
         wの0次元目は切片。1次元目以降からxの各次元に対する重みが入る。
      y_hat: ロジスティック回帰の出力。スカラー値
  '''
  # xの先頭に1(切片に相当)を加えたベクトルを作成
  # numpy append 関数：ベクトル同士をくっつけて一つのベクトルにする。
  x_mod = np.append(1, x)

  # z = w0 + w1*x1 + w2*x2 + ...
  z = np.dot(w, x_mod)

  # シグモイド関数に通す
  y_hat = sigmoid(z)

  return y_hat

例として5番目のサンプルデータに対してロジスティック回帰を行い，クラス識別を行います。  
ロジスティック回帰の出力$\hat{y}$は0～1の範囲の連続値ですので，$\hat{y} > 0.5$の場合はクラス1，$\hat{y} \leq 0.5$の場合はクラス0と識別します。

In [ ]:
# 5番目のデータで試す
sample_id = 5
w = initial_w.copy()
x = X[sample_id,:]
y = Y[sample_id]

# ロジスティック回帰関数に与える
y_hat = logistic_regression(x, w)

# y_hatが0.5を超える場合はクラス1，以下の場合はクラス0と判定する
if y_hat > 0.5:
  pred = 1
else:
  pred = 0

print('y_hat = ' + str(y_hat))
print('estimated label = ' + str(pred))
print('true label = ' + str(y))

# データをプロット
x1 = np.linspace(-3, 3)
x2 = -1.0 * (w[0] + w[1]*x1) / w[2]
plt.figure(figsize=(7,7))
plt.plot(x1, x2)
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class1')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class2')
plt.scatter(x[0], x[1], c='g')
plt.scatter(x[0], x[1], c='g', marker='x', s=100, label='tested sample')
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()


estimated label（識別結果）は1で，true label(正解ラベル)は0なので，この識別結果は誤りです。  

次に，このときの勾配を計算してみます。

まず，勾配を計算する関数を以下に定義します。    
$\frac{\partial L_{mse}}{\partial \bf w} = (\hat{y} - y)(1 - \hat{y})\hat{y}[1, x_1, x_2, \dots]$

In [ ]:
def calc_gradient(y_hat, y, x):
  '''
      勾配を計算する
      y_hat: ロジスティック回帰によって推定された y
      y: 正解のy
      x: データ
  '''
  # xの先頭に1(切片に相当)を加えたベクトルを作成
  # numpy append 関数：ベクトル同士をくっつけて一つのベクトルにする。
  x_mod = np.append(1, x)

  gradient = (y_hat - y)*(1 - y_hat)*y_hat * x_mod

  return gradient

定義した関数を使って，勾配を計算します。

In [ ]:
grad = calc_gradient(y_hat, y, x)
print('gradient of L for w = ' + str(grad))

計算された勾配を使って，
${\bf w}_{\rm next} = {\bf w} - \mu \frac{\partial L_{mse}}{\partial \bf w}$により，${\bf w}$を更新してみます。  
ここでは，$\mu = 0.2$とします。

In [ ]:
# 学習率(lr = learning rate)
lr = 0.2

# 勾配を使って w を更新
w_next = w - lr * grad

# プロット
x1 = np.linspace(-3, 3)
x2next = -1.0 * (w_next[0] + w_next[1]*x1) / w_next[2]
plt.figure(figsize=(7,7))
plt.plot(x1, x2, label='initial boundary')
plt.plot(x1, x2next, label='updated boundary')
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class1')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class2')
plt.scatter(x[0], x[1], c='g')
plt.scatter(x[0], x[1], c='g', marker='x', s=100, label='tested sample')
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

ほんの少しですが，境界線の傾きが負の方向に更新されました。  

更新前と更新後のロジスティック回帰の出力$\hat{y}$を見比べてみましょう。  


In [ ]:
y_hat_old = logistic_regression(x, w)
y_hat_next = logistic_regression(x, w_next)
print('y_hat_old = ' + str(y_hat_old))
print('y_hat_next = ' + str(y_hat_next))

更新により，ロジスティック回帰の出力がほんの少し小さくなった，つまり正解ラベル0にほんの少し近くなった，ということが分かります。

では，全てのデータに対してロジスティック回帰および勾配の計算を行い，サンプル毎に$\bf w$の更新を行いましょう。  

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

# 学習率
lr = 0.2

# 重みの初期化
w = initial_w.copy()

# 描画
fig = plt.figure(figsize=(7,7))
# データの描画
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class1')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class2')
images = []

# 全サンプルを読み込んで，サンプル毎に更新を行う。
for n in range(num_samples):
  x = X[n,:]
  y = Y[n]
  
  # ロジスティック回帰関数に与える
  y_hat = logistic_regression(x, w)
  
  # 勾配の計算
  grad = calc_gradient(y_hat, y, x)

  # 毎回アニメーションを更新すると時間がかかるので，4サンプル毎にアニメーションを更新する。
  if n % 4 == 0:
    x2 = -1.0 * (w[0] + w[1]*x1) / w[2]
    img = plt.plot(x1, x2, c='b')
    images.append(img)

  # 更新
  w -= lr * grad

plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=100)
# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)

理想的な分離境界に近づいてきましたが，まだ完全に分離できていないまま，全データが処理されてしまいました。  

そこで，データをシャッフルし直して，再度全データを読み込み，$\bf w$の更新を行います。  
ここで，全データを読み込んで更新するまでの処理1週分を<font color='red'>**エポック**</font>と呼びます。  
以下の処理では，全データを50周，つまり50エポック分，$\bf w$の更新を行います。

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

np.random.seed(0)

# 学習率
lr = 0.2

# 重みの初期化
w = initial_w.copy()

# 描画
fig = plt.figure(figsize=(7,7))
# データの描画
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class1')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class2')
images = []

# yの誤差の履歴
mse_history = np.array([])

for epoch in range(50):
  
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  
  mse = 0
  for n in range(num_samples):
    x = X_tmp[n,:]
    y = Y_tmp[n]
  
    # ロジスティック回帰関数に与える
    y_hat = logistic_regression(x, w)
  
    # 勾配の計算
    grad = calc_gradient(y_hat, y, x)

    # 毎回アニメーションを更新すると時間がかかるので，200サンプル毎にアニメーションを更新する。
    if n % 400 == 0:
      x2 = -1.0 * (w[0] + w[1]*x1) / w[2]
      img = plt.plot(x1, x2, c='b')
      img.append(plt.text(-2.8, 2.5, 'epoch: '+str(epoch), size='x-large'))
      images.append(img)

    # 更新
    w -= lr * grad

    # yの誤差を蓄積
    mse += (y - y_hat)**2
  
  mse /= num_samples
  mse_history = np.append(mse_history, mse)
  print('%d-th epoch: mean square error = %.3f' % (epoch+1, mse))

plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=100)
# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)

mean square error は 正解の$y$とロジスティック回帰が出力した$\hat{y}$の平均二乗誤差  
${\rm MSE} = \frac{1}{N}\sum_{n=1}^{N}(y_n - \hat{y}_{n})^2$  
です。エポック毎のMSEをプロットしてみます。

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(mse_history)
plt.xlabel('epoch')
plt.ylabel('mean square error')
plt.show()

更新が進むにつれて，$y$と$\hat{y}$の誤差が小さくなっているのが分かります。

最後に，分類正解率を計算します。  

In [ ]:
# 正解率
accuracy = 0

for n in range(num_samples):
  x = X[n,:]
  y = Y[n]
  # ロジスティック回帰関数に与える
  y_hat = logistic_regression(x, w)

  if y_hat > 0.5 and y == 1:
    # ロジスティック回帰出力が0.5より大きく，かつ正解ラベルが1
    accuracy += 1
  elif y_hat <= 0.5 and y == 0:
    # ロジスティック回帰出力が0.5以下，かつ正解ラベルが0
    accuracy += 1

accuracy = 100.0 * accuracy / num_samples

print('Accuracy = %.3f' % (accuracy))

ロジスティック回帰でも100%の正解率が得られました。